---
title: "Extracción de Indicadores de Banco de Información Económica"
author: "Jesús Eduardo Siqueiros"
output: 
  html_document:
    code_folding: hide
    toc: true
    toc_depth: 3
    toc_float: true
jupyter: python3
---

#### Extraccion de PIB por actividad economica para cada Estado de la Republica Méxicana.

Utilizando la API del Banco de Información Económica, extraeremos el Producto Interno Bruto (PIB) por actividad económica para cada uno de los estados de la República Mexicana. El objetivo es analizar el crecimiento individual de cada estado. Además, 

implementaremos modelos de inteligencia artificial (IA) para realizar un análisis más exhaustivo y evaluar la capacidad actual de los modelos de lenguaje a gran escala (LLM) en la comprensión del comportamiento de la información.

#### Librerias

In [10]:
# Importar datos
import requests

# Manipulacion de datos
import pandas as pd
import geopandas as gpd

# Lectura de archivos config
import configparser as cfg

# Para manejar el tiempo
import time

# Manejar rutas
import os
import shutil

# Obtener secret_key
from dotenv import load_dotenv

In [3]:
# Load environment variables from .env file
load_dotenv()

# Access the secret API key
bie_api = os.getenv('SECRET_TOKEN')

Estas librerias seran nuestras herramientas que utilizaremos para poder extraer nuestra informacion y hacer las transformaciones pertinentes a nuestro marco de datos

In [4]:
# Nombre de los estados
states = [
    "Aguascalientes",
    "Baja California",
    "Baja California Sur",
    "Campeche",
    "Coahuila de Zaragoza",
    "Colima",
    "Chiapas",
    "Chihuahua",
    "Ciudad de México",
    "Durango",
    "Guanajuato",
    "Guerrero",
    "Hidalgo",
    "Jalisco",
    "México",
    "Michoacán de Ocampo",
    "Morelos",
    "Nayarit",
    "Nuevo León",
    "Oaxaca",
    "Puebla",
    "Querétaro",
    "Quintana Roo",
    "San Luis Potosí",
    "Sinaloa",
    "Sonora",
    "Tabasco",
    "Tamaulipas",
    "Tlaxcala",
    "Veracruz de Ignacio de la Llave",
    "Yucatán",
    "Zacatecas"
]

# Crear diccionario para cada estado con las claves especificadas
states_dict = {state: {"Actividad Primaria": None, "Actividad Secundaria": None, "Actividad Terciaria": None} for state in states}

# Agrega el codigo correspondiente a cada estado
for code, state in zip(range(756304, 757649, 42), states):
    states_dict[state]["Actividad Primaria"] = code
    states_dict[state]["Actividad Secundaria"] = states_dict[state]["Actividad Primaria"] + 6
    states_dict[state]["Actividad Terciaria"] = states_dict[state]["Actividad Secundaria"] + 19

Se define el nombre de cada uno de los Estados de la republica mexciana en una `list` para posteriormete meter genera un diccionario con:

* Nombre del Estado de la republica.
    1. Actividad Primaria
    2. Actividad Secundaria 
    3. Actividad Terciaria

La información que se almacena en cada una de las llaves es el código con el que se accede a la información desde la API del BIE.

In [5]:
# Specify the directory path
directory_path = 'data'

# Delete the directory if it exists, along with its contents
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)
    print(f"Directory '{directory_path}' has been deleted.")

# Create the directory if it does not exist
os.makedirs(directory_path, exist_ok=True)
print(f"Directory '{directory_path}' has been created.")


Directory 'data' has been created.


#### Actividad Primaria

In [6]:
# Si la ruta existe, borrala para almacenar nuevos datos
if os.path.exists("data/01_primaria.csv") == True:
    os.remove("data/01_primaria.csv")

for i in states_dict.keys():
    url_primary =  f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{states_dict[i]["Actividad Primaria"]}/es/0700/true/BIE/2.0/{bie_api}?type=json"
    responses_primary = requests.get(url_primary)
    responses_primary.raise_for_status() # En caso de que la api falle, arroja un error

    # Conversion de json a pd.DataFrame
    data = responses_primary.json()["Series"]
    df = pd.json_normalize(data, "OBSERVATIONS")

    # Seleccion de columnas y renombre
    df = df[["OBS_VALUE", "TIME_PERIOD"]].rename(columns={"OBS_VALUE": "Actividad_primaria",
                                                          "TIME_PERIOD": "Fecha"})
    # Se agrega el nombre del Estado
    df["Estado"] = i

    # Almacenamiento de informacion
    if os.path.exists("data/01_primaria.csv") == True:
        df.to_csv("data/01_primaria.csv", header=False, index=False, mode="a")
    else:
        df.to_csv("data/01_primaria.csv", header=True, index=False, mode="a")

    #Puntos de control
    print(f"Estado de {i} almacenado correctamente")

    # Evitar rebotes de la API
    time.sleep(5)

Estado de Aguascalientes almacenado correctamente
Estado de Baja California almacenado correctamente
Estado de Baja California Sur almacenado correctamente
Estado de Campeche almacenado correctamente
Estado de Coahuila de Zaragoza almacenado correctamente
Estado de Colima almacenado correctamente
Estado de Chiapas almacenado correctamente
Estado de Chihuahua almacenado correctamente
Estado de Ciudad de México almacenado correctamente
Estado de Durango almacenado correctamente
Estado de Guanajuato almacenado correctamente
Estado de Guerrero almacenado correctamente
Estado de Hidalgo almacenado correctamente
Estado de Jalisco almacenado correctamente
Estado de México almacenado correctamente
Estado de Michoacán de Ocampo almacenado correctamente
Estado de Morelos almacenado correctamente
Estado de Nayarit almacenado correctamente
Estado de Nuevo León almacenado correctamente
Estado de Oaxaca almacenado correctamente
Estado de Puebla almacenado correctamente
Estado de Querétaro almacenado

#### Actividad Secundaria

In [7]:
df_actividad_secundaria= pd.DataFrame() # Se define un dataframe vacio para almacenar la informacion

# Si la ruta existe, borrala para almacenar nuevos datos
if os.path.exists("data/02_secundaria.csv") == True:
    os.remove("data/02_secundaria.csv")

for i in states_dict.keys():
    url_secundary =  f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{states_dict[i]["Actividad Secundaria"]}/es/0700/true/BIE/2.0/{bie_api}?type=json"
    responses_secundary = requests.get(url_secundary)
    responses_secundary.raise_for_status() # En caso de que la api falle, arroja un error

    # Conversion de json a pd.DataFrame
    data = responses_secundary.json()["Series"]
    df = pd.json_normalize(data, "OBSERVATIONS")

    # Seleccion de columnas y renombre
    df = df[["OBS_VALUE", "TIME_PERIOD"]].rename(columns={"OBS_VALUE": "Actividad_secundaria",
                                                          "TIME_PERIOD": "Fecha"})
    # Se agrega el nombre del Estado
    df["Estado"] = i

    # Almacenamiento de informacion
    if os.path.exists("data/02_secundaria.csv") == True:
        df.to_csv("data/02_secundaria.csv", header=False, index=False, mode="a")
    else:
        df.to_csv("data/02_secundaria.csv", header=True, index=False, mode="a")

    #Puntos de control
    print(f"Estado de {i} almacenado correctamente")

    # Evitar rebotes de la API
    time.sleep(5)

Estado de Aguascalientes almacenado correctamente
Estado de Baja California almacenado correctamente
Estado de Baja California Sur almacenado correctamente
Estado de Campeche almacenado correctamente
Estado de Coahuila de Zaragoza almacenado correctamente
Estado de Colima almacenado correctamente
Estado de Chiapas almacenado correctamente
Estado de Chihuahua almacenado correctamente
Estado de Ciudad de México almacenado correctamente
Estado de Durango almacenado correctamente
Estado de Guanajuato almacenado correctamente
Estado de Guerrero almacenado correctamente
Estado de Hidalgo almacenado correctamente
Estado de Jalisco almacenado correctamente
Estado de México almacenado correctamente
Estado de Michoacán de Ocampo almacenado correctamente
Estado de Morelos almacenado correctamente
Estado de Nayarit almacenado correctamente
Estado de Nuevo León almacenado correctamente
Estado de Oaxaca almacenado correctamente
Estado de Puebla almacenado correctamente
Estado de Querétaro almacenado

#### Actividad Terciaria

In [8]:
# Si la ruta existe, borrala para almacenar nuevos datos
if os.path.exists("data/03_terciaria.csv") == True:
    os.remove("data/03_terciaria.csv")

for i in states_dict.keys():
    url_tertiary =  f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{states_dict[i]["Actividad Terciaria"]}/es/0700/true/BIE/2.0/{bie_api}?type=json"
    responses_tertiary = requests.get(url_tertiary)
    responses_tertiary.raise_for_status() # En caso de que la api falle, arroja un error

    # Conversion de json a pd.DataFrame
    data = responses_tertiary.json()["Series"]
    df = pd.json_normalize(data, "OBSERVATIONS")

    # Seleccion de columnas y renombre
    df = df[["OBS_VALUE", "TIME_PERIOD"]].rename(columns={"OBS_VALUE": "Actividad_terciaria",
                                                          "TIME_PERIOD": "Fecha"})
    # Se agrega el nombre del Estado
    df["Estado"] = i

    # Almacenamiento de informacion
    if os.path.exists("data/03_terciaria.csv") == True:
        df.to_csv("data/03_terciaria.csv", header=False, index=False, mode="a")
    else:
        df.to_csv("data/03_terciaria.csv", header=True, index=False, mode="a")

    #Puntos de control
    print(f"Estado de {i} almacenado correctamente")

    # Evitar rebotes de la API
    time.sleep(5)

Estado de Aguascalientes almacenado correctamente
Estado de Baja California almacenado correctamente
Estado de Baja California Sur almacenado correctamente
Estado de Campeche almacenado correctamente
Estado de Coahuila de Zaragoza almacenado correctamente
Estado de Colima almacenado correctamente
Estado de Chiapas almacenado correctamente
Estado de Chihuahua almacenado correctamente
Estado de Ciudad de México almacenado correctamente
Estado de Durango almacenado correctamente
Estado de Guanajuato almacenado correctamente
Estado de Guerrero almacenado correctamente
Estado de Hidalgo almacenado correctamente
Estado de Jalisco almacenado correctamente
Estado de México almacenado correctamente
Estado de Michoacán de Ocampo almacenado correctamente
Estado de Morelos almacenado correctamente
Estado de Nayarit almacenado correctamente
Estado de Nuevo León almacenado correctamente
Estado de Oaxaca almacenado correctamente
Estado de Puebla almacenado correctamente
Estado de Querétaro almacenado

In [ ]:
# Extracion de datos geospaciales de Mexico usando geopandas
if os.path.exists("data/geo_data.geo"):
    pass
else:
    geo_data = gpd.read_file("https://gist.githubusercontent.com/walkerke/76cb8cc5f949432f9555/raw/363c297ce82a4dcb9bdf003d82aa4f64bc695cf1/mx.geojson") # Extraccion
    geo_data.to_csv("data/geo_data.geojson", header=True, index=False) # Guardado de datos

Una vez que los datos hayan sido extraidos y almacenados en archivos CSV, podemos avanzar a la siguente seccion, en la cual, se comenzará con el analisis 